In [2]:
import numpy as np 
import pandas as pd
import requests
import xlsxwriter
import math

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks


,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [4]:
from sec import IEX_CLOUD_API_TOKEN
print (IEX_CLOUD_API_TOKEN)


pk_9a92a8d1eab441dea7c7e8a25534e765


In [5]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 56490765,
 'calculationPrice': 'iexlasttrade',
 'change': -1.68,
 'changePercent': -0.00968,
 'close': None,
 'closeSource': 'official',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': None,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 171.82,
 'iexCloseTime': 1683662399009,
 'iexLastUpdated': 1683662399009,
 'iexMarketPercent': 0.011343503446518409,
 'iexOpen': 173.08,
 'iexOpenTime': 1683639000004,
 'iexRealtimePrice': 171.82,
 'iexRealtimeSize': 1,
 'iexVolume': 513868,
 'lastTradeTime': 1683662399977,
 'latestPrice': 171.82,
 'latestSource': 'IEX Last Trade',
 'latestTime': 'May 9, 2023',
 'latestUpdate': 1683662399009,
 'latestVolume': None,
 'low': None,
 'lowSource': None,
 'lowTime': None,
 'm

In [6]:
latest_price = data['iexRealtimePrice']
market_cap = data['marketCap']
print(latest_price)
market_cap

171.82


2702505577640

In [7]:
my_columns = ['Ticker', 'Stock Price','Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number Of Shares to Buy


In [8]:
final_dataframe.append(
    pd.Series(
        [
            symbol,
            data['iexRealtimePrice'],
            data['marketCap'],
            'N/A'
        ],
    index = my_columns
    ),
ignore_index = True
)

,Ticker,Stock Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,171.82,2702505577640,N/A


In [9]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker'][:200]:
    api_url = f'https://cloud.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
            [
                stock,
                data['iexRealtimePrice'],
                data['marketCap'],
                'N/A'
            ],
        index = my_columns
        ),
        ignore_index = True
        )


In [10]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number Of Shares to Buy
0,A,127.38,37657630980,N/A
1,AAL,14.465,9443656178,N/A
2,AAP,125.5,7440044991,N/A
3,AAPL,171.82,2702505577640,N/A
4,ABBV,146.81,259050653714,N/A
...,...,...,...,...
195,FTNT,66.395,52058081258,N/A
196,FTV,64.54,22814577241,N/A
197,GD,211.27,57939678931,N/A
198,GE,101,109984962929,N/A


In [11]:
# Function sourced from
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [13]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#    print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                ],
            index = my_columns),
                ignore_index =True,
            )
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number Of Shares to Buy
0,A,127.350,37657630980,N/A
1,AAL,14.465,9443656178,N/A
2,AAP,125.520,7440044991,N/A
3,AAPL,171.820,2702505577640,N/A
4,ABBV,146.830,259050653714,N/A
...,...,...,...,...
95,CINF,105.510,16587512821,N/A
96,CL,80.640,66896379567,N/A
97,CLX,168.130,20784823090,N/A
98,CMA,36.290,4778299256,N/A


In [19]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print("That's not a number! \nPlease try again:")
    portfolio_size = input('Enter the value of yoru portfolio')
    val = float(portfolio_size)


1000000.0


In [27]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Stock Price'][i])
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number Of Shares to Buy
0,A,127.350,37657630980,78
1,AAL,14.465,9443656178,691
2,AAP,125.520,7440044991,79
3,AAPL,171.820,2702505577640,58
4,ABBV,146.830,259050653714,68
...,...,...,...,...
95,CINF,105.510,16587512821,94
96,CL,80.640,66896379567,124
97,CLX,168.130,20784823090,59
98,CMA,36.290,4778299256,275


In [28]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [29]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [34]:
writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Stock Price', string_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)

0